# HPPC Parameter Extraction (Full)

The logic in this notebook is almost identical to that of `extract_hppc_partial.py`. 



In [1]:
import pandas as pd
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
from data_tools.query import DBClient 
from data_tools.collections import TimeSeries
import datetime

from dotenv import load_dotenv

load_dotenv()

ERROR:data_tools.query.postgresql_query:Could not find a POSTGRESQL_USERNAME in .env!
ERROR:data_tools.query.postgresql_query:Could not find a POSTGRESQL_PASSWORD in .env!
ERROR:data_tools.query.postgresql_query:Could not find a POSTGRESQL_DATABASE in .env!
ERROR:data_tools.query.postgresql_query:Could not find a POSTGRESQL_ADDRESS in .env!


True

In [2]:
client = DBClient(influxdb_token="s4Z9_S6_O09kDzYn1KZcs7LVoCA2cVK9_ObY44vR4xMh-wYLSWBkypS0S0ZHQgBvEV2A5LgvQ1IKr8byHes2LA==", influxdb_org="8a0b66d77a331e96")

start = datetime.datetime.fromisoformat("2025-04-11T00:00:00Z")
stop = datetime.datetime.fromisoformat("2025-04-14T00:00:00Z")

pack_voltage = client.query_time_series(start=start, stop=stop, field="TotalPackVoltage", bucket="CAN_log", granularity=0.1, units="V", car="Brightside")
pack_current = client.query_time_series(start=start, stop=stop, field="PackCurrent", bucket="CAN_log", granularity=0.1, units="A", car="Brightside")

pack_current_aligned, _ = TimeSeries.align(pack_current, pack_voltage)

Creating client with API Token: s4Z9_S6_O09kDzYn1KZcs7LVoCA2cVK9_ObY44vR4xMh-wYLSWBkypS0S0ZHQgBvEV2A5LgvQ1IKr8byHes2LA==
Creating client with Org: 8a0b66d77a331e96


In [13]:
fig, ax = plt.subplots()

ax2 = ax.twinx()
# ax.plot(pack_current.datetime_x_axis, pack_current)
ax2.plot(pack_voltage.datetime_x_axis, pack_voltage)
ax2.set_title("Pack Voltage during HPPC test")
ax2.set_ylabel("Pack Voltage (V)")
ax2.set_xlabel("Time (?)")
plt.tight_layout()
plt.show()

<IPython.core.display.Javascript object>

# Offline Parameter Identification 

For each pulse, we need to manually identify when the zero-state reponse (behaviour due to capacitor) is occuring.

In [14]:
pulse_indices = [421210, 379410, 329816, 284051, 238416, 192883, 148060, 100789, 56830, 8274]
beginning_indices = [11024, 57700, 102089, 148200, 193033, 238456, 284061, 329916, 380010, 394210]
ending_indices = [11274, 57900, 102489, 148460, 193283, 238716, 284261, 330146, 380110, 405210]

## Pulse Examination

In [68]:
fig.clear()
fig, ax = plt.subplots()
ax2 = ax.twinx()

beginning_index = 238400
end_index = 239200

pack_voltage_extracted = pack_voltage[beginning_index:end_index]
pack_current_extracted = pack_current[beginning_index:end_index]

A_index = 71
B_index = 74
C_index = 249
D_index = 254
E_index = 500

ax.plot(pack_voltage_extracted)
ax.axvline(x=A_index, color='tab:red')
ax.axvline(x=B_index, color='tab:orange')
ax.axvline(x=C_index, color='tab:green')
ax.axvline(x=D_index, color='tab:blue')
ax.axvline(x=E_index, color='red')

plt.show()

<IPython.core.display.Javascript object>

In [70]:
fig.clear()
fig, ax = plt.subplots()

ax.plot(pack_current_extracted)
ax.axvline(x=A_index, color='tab:red')
ax.axvline(x=B_index, color='tab:orange')
ax.axvline(x=C_index, color='tab:green')
ax.axvline(x=D_index, color='tab:blue')
ax.axvline(x=E_index, color='red')

plt.show()

<IPython.core.display.Javascript object>

What we are basically doing here is finding the resistance by looking at the voltage DIFFERENCE compared to the current DIFFERENCE from going from 0A to 40A and 40A back down to 0A. That's why we are taking the last and first element of the current, respectively (what happens between isn't a concern to us).

In [71]:
U_A = pack_voltage_extracted[A_index]
U_B = pack_voltage_extracted[B_index]
U_C = pack_voltage_extracted[C_index]
U_D = pack_voltage_extracted[D_index]

I_AB = pack_current_extracted[A_index:B_index]
R_0_AB = (U_A - U_B) / I_AB[-1]

I_CD = pack_current_extracted[C_index:D_index]
R_0_DC = (U_D - U_C) / I_CD[0]

R_0 = (R_0_AB + R_0_DC) / 2
U_oc = U_A

print(f"R_0: {R_0:.4f} Ohms")
print(f"U_oc: {U_oc:.1f} V")

R_0: 0.0640 Ohms
U_oc: 111.8 V


In [114]:
def BC_func(t, _U_oc, _I, _R_0, _R_P1, _C_P1, _U_discharge):
    _a = _U_oc - _I * _R_0 - _U_discharge
    _b = _I * _R_P1 
    _c = _R_P1 * _C_P1
    return _a - _b * (1 - np.exp(-t / _c))

def DE_func(t, _U_oc, _R_0, _R_P, _C_P, _U_P, _U_discharge):
    _a = _U_oc - _U_discharge
    _b = _U_P
    _c = _R_P * _C_P
    return _a - _b * np.exp(-t / _c)

In [115]:
def objective(t, _R_0, _R_P, _C_P, _C_P_2, _U_P, _U_discharge):
    _U_oc = pack_voltage_extracted[0]
    _I = pack_current_extracted
    initial_time_BC = _I.x_axis[B_index]
    initial_time_DE = _I.x_axis[D_index]
    discharge_time = _I.x_axis[C_index] - _I.x_axis[B_index]
    voltage = []
    next_voltage = 42
    
    t = t.astype(int)
    
    for i in t:
        if i <= A_index:
            next_voltage = _U_oc
        
        if A_index < i <= B_index:
            next_voltage = _U_oc - _I[i] * _R_0
        
        if B_index < i <= C_index:
            _t = _I.x_axis[i] - initial_time_BC
            time_factor = (_I.x_axis[i] - initial_time_BC) / discharge_time
            discharged_voltage = time_factor * _U_discharge

            next_voltage = BC_func(_t, _U_oc, _I[i], _R_0, _R_P, _C_P, discharged_voltage)
        
        if C_index < i <= D_index:
            next_voltage = _U_oc - _U_P - _U_discharge - _I[i] * _R_0
        
        if i > D_index:
            _t = _I.x_axis[i] - initial_time_DE
            next_voltage = DE_func(_t, _U_oc, _R_0, _R_P, _C_P_2, _U_P, _U_discharge)
            
        voltage.append(next_voltage)
        
    return voltage

In [116]:
from scipy import optimize 
from sklearn.preprocessing import MinMaxScaler


# Original (non-normalized) bounds
lower_bounds = np.array([
    0.01,  # R_0_data
    0.01,  # R_P_data
    10.0,  # C_P_data
    0.0,   # C_P_2_data
    0.01,  # U_P_data
    0.01   # U_discharged_data
])
upper_bounds = np.array([
    0.5,     # R_0_data
    0.5,     # R_P_data
    1000.0,  # C_P_data
    2000.0,  # C_P_2_data
    15.0,    # U_P_data
    15.0,    # U_discharged_data
])

bounds = list(zip(lower_bounds, upper_bounds))

# Create a scaler to map [lower_bounds, upper_bounds] <-> [0, 1]
scaler = MinMaxScaler()
scaler.fit(np.vstack([lower_bounds, upper_bounds]))  # shape (2, n_features)
scaled_bounds = scaler.transform([lower_bounds, upper_bounds])
scaled_lower_bounds = scaled_bounds[0]
scaled_upper_bounds = scaled_bounds[1]

# --- Scaled objective function ---
def scaled_objective(t, *scaled_params):
    real_params = scaler.inverse_transform([scaled_params])[0]
    return objective(t, *real_params)

# Idk, seems okay to me
p0 = [R_0, R_0, 300., 300., 2.0, 0.3]
# --- Define initial guess and scale it ---
p0_scaled = scaler.transform([p0])[0]

# Oh, the glory of `TimeSeries`!
x_data = np.arange(len(pack_voltage_extracted))
y_data = pack_voltage_extracted

objective(x_data, *p0)

x_scaled, pcov = optimize.curve_fit(
    scaled_objective, 
    x_data, 
    y_data, 
    p0=p0_scaled, 
    maxfev=4000,
    bounds=(scaled_lower_bounds, scaled_upper_bounds),
)
x = scaler.inverse_transform([x_scaled])[0]

fig, ax = plt.subplots()

ax.plot(objective(x_data, *x), label="Predicted")
ax.plot(pack_voltage_extracted, label="Measured")
ax.set_title("Fitting ECM to HPPC pulse data")
ax.set_xlabel("Time (10ms)")
ax.set_ylabel("Voltage (V)")
plt.legend()
plt.show()

<IPython.core.display.Javascript object>

In [117]:
pcov.diagonal()

array([8.12150633e-06, 4.76151537e-05, 4.61935903e-03, 2.14412783e-02,
       1.18313646e-05, 1.68526352e-05])

In [118]:
[print(f"{_x:.4f}") for _x in x]

0.1084
0.0268
338.4230
938.0481
0.7806
0.4560


[None, None, None, None, None, None]

In [12]:
R_0_data = [0.1807, 0.1413, 0.1254, 0.1146, 0.1056, 0.1068]
R_P_data = [0.1935, 0.1574, 0.1983, 0.1651, 0.1931, 0.1459]
C_P_data = [197.8862, 288.7784, 246.6670, 326.9496, 269.9395, 348.3759]
U_oc_data = [131.6941, 129.4818, 125.8064, 122.0371, 118.3019, 115.5432]

raw_params = f"""
R_0_data = {[float(x) for x in R_0_data]}
R_P_data = {[float(x) for x in R_P_data]}
C_P_data = {[float(x) for x in C_P_data]}
Uoc_data = {[float(x) for x in U_oc_data]}
"""

print(raw_params)


R_0_data = [0.1413]
R_P_data = [0.1574]
C_P_data = [288.7784]
Uoc_data = [129.4818]

